**Created And Trained By Muhammad Hanan Asghar**

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Mounted at /gdrive
/gdrive


In [ ]:
import zipfile
zip_ref = zipfile.ZipFile('/gdrive/MyDrive/datasetsofhandsimages.zip','r')
zip_ref.extractall("/content")
zip_ref.close()

In [ ]:
import os

In [ ]:
BASE_DIR = '/content/datasets'

TRAIN_DIR = os.path.join(BASE_DIR, 'train')
TEST_DIR = os.path.join(BASE_DIR, 'test')

In [ ]:
!rm -rf /content/output
!mkdir /content/output

In [ ]:
!pip -q install keras-tuner

     |████████████████████████████████| 71kB 7.1MB/s 


In [ ]:
import keras
from keras.layers import Conv2D, MaxPooling2D, Flatten
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping
from kerastuner import RandomSearch

In [ ]:
def Model(hp):
  cnn = Sequential()
  cnn.add(Conv2D(filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
                 kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
                 input_shape=(64, 64, 3), 
                 activation='relu')) 
  
  cnn.add(MaxPooling2D(pool_size=(2, 2)))
   
  cnn.add(Conv2D(filters=hp.Int('conv_2_filter', min_value=32, max_value=128, step=16),
                 kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
                 activation='relu'))
  
  cnn.add(MaxPooling2D(pool_size=(2, 2)))
  
  cnn.add(Flatten())

  cnn.add(Dense(units=hp.Int('dense_1_units', 
                min_value=64, max_value=512, step=16),
                activation='relu'))
  
  cnn.add(Dropout(rate=0.5))

  cnn.add(Dense(units=5, activation='softmax'))

  cnn.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='categorical_crossentropy',
              metrics=['accuracy'])
  return cnn

In [ ]:
tuner = RandomSearch(Model,
                    objective='val_accuracy',
                    max_trials=4,
                    directory='/content/output',
                    project_name="Hands Digits Detection")

In [ ]:
TRAIN_DIR

'/content/datasets/train'

In [ ]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        TRAIN_DIR,
        target_size=(64, 64),
        batch_size=25,
        class_mode='categorical')

validation_generator = test_datagen.flow_from_directory(
        TEST_DIR,
        target_size=(64, 64),
        batch_size=25,
        class_mode='categorical')

Found 1915 images belonging to 5 classes.
Found 257 images belonging to 5 classes.


In [ ]:
train_generator.class_indices

{'1': 0, '2': 1, '3': 2, '4': 3, '5': 4}

In [ ]:
tuner.search(train_generator,
              validation_data = validation_generator,epochs=5)

Trial 4 Complete [00h 00m 19s]
val_accuracy: 0.26459142565727234

Best val_accuracy So Far: 1.0
Total elapsed time: 00h 01m 20s
INFO:tensorflow:Oracle triggered exit


In [ ]:
model=tuner.get_best_models(num_models=1)[0]

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 60, 60, 96)        7296      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 30, 30, 96)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 28, 28, 80)        69200     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 80)        0         
_________________________________________________________________
flatten (Flatten)            (None, 15680)             0         
_________________________________________________________________
dense (Dense)                (None, 112)               1756272   
_________________________________________________________________
dropout (Dropout)            (None, 112)               0

In [ ]:
model.fit(train_generator,
          validation_data = validation_generator,
          epochs=10, initial_epoch=3)

Epoch 4/10
77/77 [==============================] - 4s 50ms/step - loss: 0.0338 - accuracy: 0.9911 - val_loss: 0.0086 - val_accuracy: 1.0000
Epoch 5/10
77/77 [==============================] - 4s 48ms/step - loss: 0.0251 - accuracy: 0.9937 - val_loss: 0.0169 - val_accuracy: 1.0000
Epoch 6/10
77/77 [==============================] - 4s 47ms/step - loss: 0.0196 - accuracy: 0.9937 - val_loss: 0.0521 - val_accuracy: 1.0000
Epoch 7/10
77/77 [==============================] - 4s 48ms/step - loss: 0.0346 - accuracy: 0.9906 - val_loss: 0.1606 - val_accuracy: 0.8988
Epoch 8/10
77/77 [==============================] - 4s 48ms/step - loss: 0.0253 - accuracy: 0.9927 - val_loss: 0.0689 - val_accuracy: 0.9728
Epoch 9/10
77/77 [==============================] - 4s 48ms/step - loss: 0.0257 - accuracy: 0.9922 - val_loss: 0.0227 - val_accuracy: 1.0000
Epoch 10/10
77/77 [==============================] - 4s 48ms/step - loss: 0.0266 - accuracy: 0.9927 - val_loss: 6.1065e-05 - val_accuracy: 1.0000


In [ ]:
!rm -rf /gdrive/MyDrive/ModelTrainedOfHands
!mkdir /gdrive/MyDrive/ModelTrainedOfHands

In [ ]:
model_json = model.to_json()
with open("/gdrive/MyDrive/ModelTrainedOfHands/model-hand-digits.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights('/gdrive/MyDrive/ModelTrainedOfHands/model-hand-digits.h5')

In [ ]:
from keras.models import model_from_json

In [ ]:
json_file = open("/gdrive/MyDrive/ModelTrainedOfHands/model-hand-digits.json", "r")
model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(model_json)
loaded_model.load_weights("/gdrive/MyDrive/ModelTrainedOfHands/model-hand-digits.h5")
print("Loaded model from disk")

Loaded model from disk


In [ ]:
import numpy as np
from keras.preprocessing import image



test_image = image.load_img("/content/6.jpg", target_size=(64,64))

test_image = image.img_to_array(test_image)

test_image = np.expand_dims(test_image, axis = 0)

In [ ]:
result = loaded_model.predict(test_image)

In [ ]:
if int(round(result[0][0])):
  print("One")
elif int(round(result[0][1])):
  print("Two")
elif int(round(result[0][2])):
  print("Three")
elif int(round(result[0][3])):
  print("Four")
elif int(round(result[0][4])):
  print("Five")

One


In [ ]:
result

array([[0., 0., 1., 0., 0.]], dtype=float32)